In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso



# evaluate an Passive agressive regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold





    
halan_regression3 = pd.read_csv('csv/weekly_data2.csv')
halan_regression3['average']=halan_regression3.iloc[:,1:5].mean(axis=1)
df = halan_regression3.copy()
df['w1']=df['w1']/df['average']
df['w2']=df['w2']/df['average']
df['w3']=df['w3']/df['average']
df['w4']=df['w4']/df['average']

columns = ['w1','w2','w3','w4']
deseasonalized = pd.DataFrame()

deseasonalized['Month']=[halan_regression3['Month'][j] for j in range(len(df)) for i in columns]
deseasonalized['Week']=[int(i[-1]) for j in range(len(df)) for i in columns]


deseasonalized['Original']=[halan_regression3[i][j] for j in range(len(df)) for i in columns]
deseasonalized['Delivered'] =[halan_regression3[i][j]/df[i].mean(axis=0) for j in range(len(df)) for i in columns]       
deseasonalized['MA12'] = deseasonalized['Original'].rolling(5).mean()# plot the data and MA
deseasonalized['Series'] = np.arange(1,len(deseasonalized)+1)



# drop unnecessary columns and re-arrange
data = deseasonalized[['Series', 'Month', 'Week', 'Original']]


x = np.array(data.drop(["Original",'Series'], 1))
y = np.array(data["Original"])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=42)
model = Lasso(alpha=1, copy_X=True,
                fit_intercept=True,
            max_iter=1000, 
            positive=False,
            precompute=False,
            random_state=123,
            selection='cyclic', tol=0.0001,
            warm_start=False)
model.fit(x, y)



# ypred = model.predict(xtest)

# pred_del = pd.DataFrame(data={"Predicted Delivered": ypred.flatten()})
# print(pred_del.head())

# cv = RepeatedKFold(n_splits=8, n_repeats=30, random_state=1)
# # define model

# # fit model




future_df = pd.DataFrame()
future_df['Month'] =  [i for i in  np.arange(1,13) for j in  np.arange(1,5)]   
future_df['Week'] = [j for i in  np.arange(1,13) for j in  np.arange(1,5)]  
future_df['Series'] = np.arange(0,(48))

predictions_future = model.predict( future_df.drop(['Series'],axis=1,))
predictions = pd.DataFrame(predictions_future, columns=['Label'])
future_df['Label']=predictions['Label']

concat_df = pd.concat([data,future_df], axis=0)
df2 = df.rename({'w1': 1, 'w2': 2,'w3':3,'w4':4}, axis=1) 
avgs=[]
for i in range(4):
    avgs.append(df2[i+1].mean())
res = concat_df.dropna(subset=['Label'], axis=0)
res.drop(['Original'], axis=1, inplace=True)
res['reseason']= res['Label']   
bias=20
lag=0
if not bias:
    bias = 1
if not lag:
    lag= 0
for i in range(len(res['reseason'])):
    res['reseason'][i]=res['reseason'][i]*(avgs[res['Week'][i]-1])*(1-lag)+bias*i
concat_res = pd.concat([res,deseasonalized], axis=0)



C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11816\1870948150.py:48: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Lasso was fitted without feature names

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11816\1870948150.py:91: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11816\1870948150.py:92: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [143]:
concat_res

,Series,Month,Week,Label,reseason,Original,Delivered,MA12
0,0,1,1,487.019301,561.692406,NaN,NaN,NaN
1,1,1,2,463.233777,442.045098,NaN,NaN,NaN
2,2,1,3,439.448253,431.821444,NaN,NaN,NaN
3,3,1,4,415.662729,389.735352,NaN,NaN,NaN
4,4,2,1,393.503248,461.837836,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
19,20,10,4,NaN,NaN,159.000000,172.229047,465.200000
20,21,11,1,NaN,NaN,461.613733,400.245393,417.522747
21,22,11,2,NaN,NaN,415.262743,437.145487,396.575295
22,23,11,3,NaN,NaN,417.597030,428.945972,372.694701


In [23]:
avgs

[1.1533267858917444,
 0.9499417347539348,
 0.9735422564942372,
 0.9231892228600836]

In [22]:

fig = px.line(concat_res, x="Series", y=["reseason",'Label',"Original"], template = 'plotly_white',
title='Lasso Regression Reseasonalized Target Prediction',
)
fig.update_layout(showlegend=True,   margin=dict(t=60,l=10,b=20,r=10),height=300,
    xaxis_title="Week Number Time Series",
    yaxis_title="Number of Delivered Orders",
        font = dict(family="Arial",),)
fig.update_traces(textposition='top center')
newnames = {'reseason':'Reseasonalized Predicted', 'Label': 'Predicted Weekly Average','Original': "Actual Weekly Average"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                legendgroup = newnames[t.name],
                                hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

fig.show()

In [111]:


xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=42)
ypred = model.predict(xtest)

pred_del = pd.DataFrame(data={"Predicted Delivered": ypred.flatten()})



print(pred_del.head())

   Predicted Delivered
0           476.242117
1           489.250457
2           463.233777


In [116]:
cv = RepeatedKFold(n_splits=4, n_repeats=30, random_state=1)
scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)
scores

array([ -43.98125664, -136.59152988,  -83.96794837, -122.99999263,
        -94.96221405, -184.43239334, -117.25099763,  -71.86321162,
        -48.96485579,  -85.59565291, -194.22639743, -103.44641852,
       -171.46500682,  -71.37148155,  -56.6796956 , -127.46239642,
       -119.8130564 , -124.35909938,  -96.66604894,  -53.09503881,
       -142.08631265,  -90.63045167,  -71.42252375,  -65.68577629,
        -55.45723007, -108.22487241, -114.81215584, -153.20145793,
        -68.88960462,  -76.87885124, -151.16700437, -157.02538946,
       -143.03947741,  -70.4815009 ,  -92.78564791, -100.98727992,
       -111.71169856, -159.94675424,  -65.41094065,  -87.2141872 ,
        -71.80731818,  -66.37677915,  -73.30636195, -174.27620649,
       -169.92321707,  -67.39852221, -127.78280808,  -75.40417292,
        -87.14389082,  -69.69286213, -111.16889242, -127.66507362,
        -70.57617275, -101.80683549, -165.38366465,  -73.79342338,
       -109.77437005,  -59.0618897 , -156.68586444,  -86.17770

In [117]:



# # evaluate model

scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 102.571 (34.927)


In [120]:
from sklearn.metrics import explained_variance_score
ypred = model.predict(x)

evs = explained_variance_score(y, ypred)

print(f'The explained variance score is {evs}')

The explained variance score is 0.05696833996584627


In [121]:
from sklearn.metrics import max_error
max_error(y, ypred)

283.7321578877385

In [122]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, ypred)

13044.533571028718

In [123]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, ypred)

86.96321371657734

In [124]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(y, ypred)

0.0826860738403451

In [125]:
from sklearn.metrics import r2_score
r2_score(y, ypred)

0.05696833996584627

In [126]:
from sklearn.metrics import mean_pinball_loss
from sklearn.metrics import make_scorer

mean_pinball_loss(y, ypred, alpha=0.5)
mean_pinball_loss_95p = make_scorer(mean_pinball_loss, alpha=0.95)
mean_pinball_loss_95p

make_scorer(mean_pinball_loss, alpha=0.95)

In [127]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

estimator = GradientBoostingRegressor(
    loss="quantile",
    alpha=0.95,
    random_state=0,
)
cross_val_score(estimator, x, y, cv=5, scoring=mean_pinball_loss_95p)

array([ 9.71002696, 14.33002696, 74.71880354, 16.4566061 , 63.82344468])

In [102]:
concat_res

,Month,Week,Label,reseason,Original,Delivered,MA12
0,1,1,431.997905,498.234755,NaN,NaN,NaN
1,1,2,409.838423,391.322623,NaN,NaN,NaN
2,1,3,387.678941,381.421831,NaN,NaN,NaN
3,1,4,365.519460,343.443626,NaN,NaN,NaN
4,2,1,438.330744,513.538588,NaN,NaN,NaN
...,...,...,...,...,...,...,...
19,10,4,NaN,NaN,159.000000,172.229047,465.200000
20,11,1,NaN,NaN,461.613733,400.245393,417.522747
21,11,2,NaN,NaN,415.262743,437.145487,396.575295
22,11,3,NaN,NaN,417.597030,428.945972,372.694701
